# Closed loop simulations

The objective in this notebook is to close the former strategy in Symuvia. 

In [1]:
import os

from ctypes import cdll, create_string_buffer, c_int, byref, c_bool, c_double

from sqlalchemy import create_engine, MetaData
from sqlalchemy import Table, Column, String, Integer, Float 
from sqlalchemy import insert, delete, select, case, and_

from xmltodict import parse
from xml.parsers.expat import ExpatError

from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display

from symuviapy.symfunc import queueveh, getlead, getspace, getleaderspeed, updatelist, typedict, check_veh_creation
from symuviapy.contfunc import compute_control, format_open_loop, solve_tactical_problem, headway_reference

#### 1. Setting Parameters

Traffic simulation parameters 

In [2]:
DT = 0.1 # Sample time 

KC = 0.16 # CAV max density 
KH = 0.0896 # HDV max density
VF = 25.0 # Speed free flow
W = 6.25 # Congestion speed 
E  = 25.0*0.3 # Speed drop for relaxation 

GCAV = 1/(KC*W) # Time headway CAV 
GHDV = 1/(KH*W) # Time headway HDV 
SCAV = VF/(KC*W)+1/KC #  Desired space headway CAV 
SHDV = VF/(KH*W)+1/KH #  Desired space headway HDV

dveh_twy = {'CAV': GCAV, 'HDV': GHDV}
dveh_dwy = {'CAV': 1/KC, 'HDV': 1/KH}

U_MAX = 1.5 # Max. Acceleration
U_MIN = -1.5 # Min. Acceleration

#### 2. Database engine configuration

Reading database files 

In [3]:
dir_path = os.getcwd()
engine_path = ('..','Output','SymOut.sqlite')
engine_name = os.path.join(os.path.sep,*engine_path)
engine_full_name = os.path.join(dir_path,*engine_path)
engine_call = 'sqlite://'+engine_name
engine = create_engine(engine_call)
metadata = MetaData()

#### 3. Open dB Connection + Table `headway`

In [4]:
if os.path.isfile(engine_full_name):
    try:
        ltbstr = 'Loaded table in: '
        connection = engine.connect()        
        traj = Table('traj', metadata, autoload=True, autoload_with=engine)        
        closed = Table('closed', metadata, autoload=True, autoload_with=engine)
        headway = Table('headway', metadata, autoload=True, autoload_with=engine)
        control = Table('control', metadata, autoload=True, autoload_with=engine)
    except:
        ltbstr = 'Created table in: '
        traj = Table('traj', metadata,
                 Column('ti', Float()),
                 Column('id', Integer()),
                 Column('type', String(3)),
                 Column('tron', String(10)),
                 Column('voie', Integer()),
                 Column('dst', Float()),
                 Column('abs', Float()),
                 Column('vit', Float()),
                 Column('ldr', Integer()),
                 Column('spc', Float()),
                 Column('vld', Float()))        
        closed = Table('closed', metadata,
                 Column('ti', Float()),
                 Column('id', Integer()),
                 Column('type', String(3)),
                 Column('tron', String(10)),
                 Column('voie', Integer()),
                 Column('dst', Float()),
                 Column('abs', Float()),
                 Column('vit', Float()),
                 Column('ldr', Integer()),
                 Column('spc', Float()),
                 Column('vld', Float()))
        control = Table('control', metadata,
                 Column('ti', Float()),
                 Column('id', Integer()),
                 Column('type', String(3)),
                 Column('tron', String(10)),
                 Column('voie', Integer()),
                 Column('ctr', Float()),
                 Column('nit', Integer())) 
        headway = Table('headway', metadata,
                 Column('ti', Float()),
                 Column('id', Integer()),
                 Column('gapt', Float()))
        metadata.create_all(engine)
        connection = engine.connect()
    finally: 
        print(ltbstr, engine)
                

Loaded table in:  Engine(sqlite:///../Output/SymOut.sqlite)


#### 3. Load traffic library

Load Symuvia library: 

In [5]:
dir_path = os.getcwd()
lib_path_name = ('..','SymuviaModPos','Contents','Frameworks','libSymuVia.dylib')
full_name = os.path.join(dir_path,*lib_path_name)
symuvialib = cdll.LoadLibrary(full_name)     

#### 4. Load traffic network 

Load traffic network 

In [6]:
file_path = ('..', 'Network', 'Merge_Demand.xml')
file_name = os.path.join(dir_path, *file_path)

# Pointers
sRequest = create_string_buffer(100000)
bEnd = c_int()
bSecond = c_bool(True)

#### 5. Widgets for evolution 

Helpful widgets to visualize progress in the simulation 

In [7]:
progressSim = widgets.FloatProgress(
    value=5,
    min=0,
    max=71.9,
    step=0.1,
    description='Simulating:',
    bar_style='info',
    orientation='horizontal'
)
tiVal = widgets.BoundedFloatText(
    value=7.5,
    min=0,
    max=80.0,
    step=0.1,
    description='Time step:',
    disabled=False
)

#### 6. Start closed loop

Within the closed loop at each time step the following activities are performed 

0. Retrieve sample
1. Format samples 
3. Check initial conditions/ Enable control
2. Allocate samples
4. Solve tactical strategy (solved once) 
3. Solve control for CAV 
4. Compute evolution with corresponding control 
5. Format data 
6. Write in dataBase 
7. Update Symuvia 
8. Go to 0.



In [16]:
def create_platoon(lVehDataFormat):
    """
        Create platoon of vehicles 
    """
    lPlatoon = {}
    roads = []
    try:
        for veh in lVehDataFormat: 
            key = veh['tron'] 
            if key in roads:
                if veh['type'] == 'CAV':            
                    lPlatoon[key].append(veh['id'])
            else: 
                roads.append(key)            
                lPlatoon[key] = [veh['id']] if veh['type']=='CAV' else []
    except TypeError:
        veh = lVehDataFormat
        lPlatoon = [veh['id']] if veh['type']=='CAV' else []
        
    return lPlatoon

def format_data_controller(lVehDataFormat, lPlatoon):
    """
        Format data at time t for the controller, and
        boundary data
    """
    roads = [] 
    control_data = defaultdict(list)
    bound_data = {}
    bBound = False
    for veh in lVehDataFormat:         
        key = veh['id'] 
        veh_data = (float(veh['ti']),
                    veh['id'],
                    veh['type'],
                    veh['tron'],
                    veh['dst'],
                    veh['abs'],
                    veh['vit'],
                    veh['ldr'],
                    veh['spc'],
                    veh['vld'],
                   )

        for road, platoon in lPlatoon.items():
            if key in platoon:
                control_data[road].append(veh_data)
                break
            bBound = True
        key_rd = veh['tron']
        if key_rd in roads:            
            bound_data[key_rd].append(veh_data) 
            bBound = False
        elif bBound: 
            roads.append(key_rd)
            bound_data[key_rd] = [veh_data]
            bBound = False
        else:
            continue
            
    control_values = list(dict(control_data).values())
    bound_values = list(bound_data.values())
    return control_values, bound_values 

In [17]:
%%time

# Simulation steps
N = 120
step = iter(range(N)) 

# Initialize simulation
m = symuvialib.SymLoadNetworkEx(file_name.encode('UTF8'))


# Widgets 
display(progressSim) 
display(tiVal)

# Constant + Initial values
bSuccess = 2
bEnableControl = False
bTacticalComputed = False 
t = []
nVehInitial = {'In_main':8,
               'In_onramp':2,
               'Merge_zone':8,
               'Out_main':8,
              }

while bSuccess>0:
    # 0. 
    bSuccess =  symuvialib.SymRunNextStepEx(sRequest, True, byref(bEnd))
    
    try: 
        
        # 1. 
        dParsed = parse(sRequest.value.decode('UTF8'))        
        ti = dParsed['INST']['@val']
        
        if dParsed['INST']['TRAJS'] is None:
            
            # Empty network             
            pass 
        
        else:
            
            lVehDataRaw = dParsed['INST']['TRAJS']['TRAJ']                  
            lVehDataFormat = []
            
            try:
                # Single veh @ ti 
                
                lVehDataFormat = typedict(lVehDataRaw)
                lVehDataFormat['ti'] = ti
                dLeader = {lVehDataFormat['tron']: [lVehDataFormat['id']]}                
                lVehDataFormat['ldr'] = getlead(dLeader, lVehDataFormat)  
                
            except TypeError:
                
                # Multiple veh @ ti
                for i,  veh in enumerate(lVehDataRaw):
                    dVehData = typedict(veh)
                    dVehData['ti'] = ti
                    dLeader = queueveh(dLeader, dVehData)
                    dVehData['ldr'] = getlead(dLeader, dVehData)
                    lVehDataFormat.append(dVehData)
                    
            lSpacing = getspace(lVehDataFormat)
            lLeaderSpeed = getleaderspeed(lVehDataFormat)
            lVehDataFormat = updatelist(lVehDataFormat,lSpacing)
            lVehDataFormat = updatelist(lVehDataFormat,lLeaderSpeed)
            
            
            if bEnableControl:
                if bTacticalComputed:
                    print('Tactics computed {}'.format(ti))
                    
                    # Allocate samples
                    control_data, bound_data = format_data_controller(lVehDataFormat, lVehCAVs)
                    
                    # Prediction horizon
                    tf = float(ti) + 5.0                    
                    bHorizon = (refDf.index >= float(ti)) & (refDf.index <= tf)                    
                    refFuture = refDf.loc[bHorizon,:]   
                    
                    for veh_data, id_platoon in zip(control_data, lVehCAVs.values()):            
                        print(veh_data,id_platoon)
                        refPlatoon = refFuture[id_platoon].as_matrix()
                        S, V, DV, U_star, DU, n = compute_control(veh_data, refPlatoon, 0) 
#                         print(refPlatoon.shape)
                    
                else: 
                    dTrigTau = solve_tactical_problem(lVehDataFormat)
                    refDf = headway_reference(dTrigTau)
                    bTacticalComputed = True       
            else:
                bEnableControl = check_veh_creation(lVehDataFormat, nVehInitial)
                lVehCAVs = create_platoon(lVehDataFormat)
        
        n = next(step)           
        t.append(ti)
        progressSim.value = ti
        tiVal.value = ti
    except StopIteration:
        print('Stop by iteration')
        print('Last simluation step at time: {}'.format(ti))
        bSuccess = 0
    except ExpatError:
        bSuccess =  symuvialib.SymRunNextStepEx(sRequest, True, byref(bEnd))
        print('Return from Symuvia Empty: {}'.format(sRequest.value.decode('UTF8')))
        print('Last simluation step at time: {}'.format(ti))
        bSuccess = 0
        

FloatProgress(value=12.0, bar_style='info', description='Simulating:', max=71.9)

BoundedFloatText(value=12.0, description='Time step:', max=80.0, step=0.1)

Tactics computed 12.10
[(12.1, 0, 'CAV', 'In_main', 271.25, -728.75, 25.0, 0, 31.25, 25.0), (12.1, 1, 'CAV', 'In_main', 240.0, -760.0, 25.0, 0, 31.25, 25.0), (12.1, 2, 'CAV', 'In_main', 208.75, -791.25, 25.0, 1, 31.25, 25.0), (12.1, 3, 'CAV', 'In_main', 177.5, -822.5, 25.0, 2, 31.25, 25.0), (12.1, 4, 'CAV', 'In_main', 146.25, -853.75, 25.0, 3, 31.25, 25.0), (12.1, 5, 'CAV', 'In_main', 115.0, -885.0, 25.0, 4, 31.25, 25.0), (12.1, 7, 'CAV', 'In_main', 83.75, -916.25, 25.0, 5, 31.25, 25.0), (12.1, 8, 'CAV', 'In_main', 52.5, -947.5, 25.0, 7, 31.25, 25.0)] [0, 1, 2, 3, 4, 5, 7, 8]


KeyError: 4

In [31]:
m = symuvialib.SymLoadNetworkEx(file_name.encode('UTF8'))
N = 800 # Simulation steps
for s in time:
    
    # For all vehicle connected if created:
    # Drive vehicle 
   # symuvialib.SymDriveVehiculeEx(Id,sTroncon, nVoie, dPos, bForce)

    bResult = symuvialib.SymRunNextStepEx(sRequest, True, byref(bEnd)) 
    dParsed = parse(sRequest.value.decode('UTF8'))
    print('Instant:',s+1)


#     print(f'bResult: {bResult}')    
#     print(f'OutputNet: {sRequest.value}')        
    ti = dParsed['INST']['@val']
    print(ti, bResult)

# # Create vehicle at time step 0.5
# stype = 'CAV'
# sOrigin= 'Ext_In_main'
# sDestination = 'Ext_Out_main'
# nVoie = c_int(1)
# dbTime = c_double(0.05)
# nIdVeh = symuvialib.SymCreateVehicleEx(stype.encode('UTF8'), sOrigin.encode('UTF8'), sDestination.encode('UTF8'), nVoie, dbTime)
# print('Vehicle created', nIdVeh)
    
    
# # Run time step 
# bResult = symuvialib.SymRunNextStepEx(sRequest, 1, byref(bEnd)) 
# dParsed = parse(sRequest.value.decode('UTF8'))
# print(f'bResult: {bResult}')    
# print(f'OutputNet: {sRequest.value}')  
# ti = dParsed['INST']['@val']
# print(ti, bResult)   

# # Drive vehicle @ 0.5

sTroncon = 'In_main'.encode('UTF8')
nVoie = c_int(1)
dPos = c_double(15.0)
Id = c_int(0)
bForce = c_int(1)
#print(sTroncon,nVoie,dPos,Id, bForce)
dParsed1 = dParsed
print(dParsed1)

nres = symuvialib.SymDriveVehicleEx(Id, sTroncon, nVoie, dPos, bForce)


bResult = symuvialib.SymRunNextStepEx(sRequest, 1, byref(bEnd)) 
dParsed = parse(sRequest.value.decode('UTF8'))
# print(f'bResult: {bResult}')    
# print(f'OutputNet: {sRequest.value}')  
ti = dParsed['INST']['@val']
print(ti, bResult)   
print(dParsed)

# print('Drive ',nres )

NameError: name 'time' is not defined

#### Reading from dB

Reading `headway` table to check reference headway times 